In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Flatten
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from tensorflow.keras.callbacks import EarlyStopping
from keras_tuner import RandomSearch
from tensorflow import keras
from keras.layers import Conv2D, MaxPooling2D, Reshape
from keras.optimizers import Adam
import keras
import random

# Load the data
data = pd.read_csv('/Users/thomas/Documents/GitHub/CNN-LSTM/Models_v2/Final_df/BTC_FINAL_df_class.csv')


# MAke the date the index
data.set_index('Date', inplace=True)

seed_value = 12
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Normalize the data
# List of columns to scale
columns_to_scale = ['Open', 'High', 'Low','Close', 'Volume', 'RSI', 'ATR', 'MACD', 'MFI',
                    'EMA', 'SMA', 'OBV', 'GTrends_Interest', 'Sentiment_Bullish',
                    'Price_oil', 'Price_gold', 'Price_NASDAQ', 'Price_SP500', 'Price_NYSE',
                    'Interest_Rate', 'hash_rate', 'users']

# Initialize the scaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Scale the selected columns
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])

# Train and test data
training_size = int(len(data) * 0.9)
training_data = data[:training_size]
test_data = data[training_size:]

train_data = training_data[:int(len(training_data) * 0.9)]
val_data = training_data[int(len(training_data) * 0.9):]


# Define the function to create the dataset
def create_dataset(data, window_size, target_index):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data.iloc[i:(i + window_size)].values)  
        y.append(data.iloc[i + window_size, target_index])
    return np.array(X), np.array(y)

# Forecast Horizon
window_size = 3

# Target index ('Close')
close_index = 22

# Create the dataset
X_train, y_train = create_dataset(train_data, window_size, close_index)
X_test, y_test = create_dataset(test_data, window_size, close_index)
X_val, y_val = create_dataset(val_data, window_size, close_index)
X_train_full, y_train_full = create_dataset(training_data, window_size, close_index)

# Reshape the data
X_train = X_train.reshape((X_train.shape[0], window_size, X_train.shape[2], 1))
X_test = X_test.reshape((X_test.shape[0], window_size, X_test.shape[2], 1))
X_val = X_val.reshape((X_val.shape[0], window_size, X_val.shape[2], 1))
X_train_full = X_train_full.reshape((X_train_full.shape[0], window_size, X_train_full.shape[2], 1))

def build_model(hp):
    model = Sequential()
    filters = hp.Int('filters', min_value=95, max_value=95, step=5)
    pool = hp.Choice('pool_size', values=[2, 3])
    model.add(Conv2D(
        filters=filters,
        kernel_size=3,
        activation='relu',
        input_shape=(X_train.shape[1], X_train.shape[2], 1),
        padding='same'
    ))
    
    model.add(MaxPooling2D(pool_size=pool, padding='same'))

    model.add(Conv2D(
        filters=filters,
        kernel_size=3,
        activation='relu',
        padding='same'
    ))
    
    model.add(MaxPooling2D(pool_size=pool, padding='same'))
  
    model.add(Dense(
        hp.Int('dense_units', min_value=115, max_value=115, step=5),
        activation='relu'))
    

    x,y = model.output_shape[2], model.output_shape[3]

    model.add(Reshape((x, y)))

    model.add(LSTM(
        units=hp.Int('lstm_units', min_value=30, max_value=30, step=10),
        activation='relu',
    ))
    model.add(Dropout(rate=hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)))
    model.add(Dense(units=1, activation='sigmoid'))
    
    model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                loss='binary_crossentropy',
                metrics=['accuracy'])

    return model

# Try 1: CNNLSTM-3D-CLASS-2 best
#TRY 2: CNNLSTM-3D-CLASS-3


# Instantiate the tuner
tuner = RandomSearch(
    build_model,
    objective='val_accuracy', 
    max_trials=200,
    executions_per_trial=1,
    directory='my_dir',
    project_name='CNNLSTM-3D-CLASS-2v2',
    overwrite=True
)


# Configure EarlyStopping
early_stopping = EarlyStopping(
    monitor='val_accuracy',   
    patience=15,          
    restore_best_weights=True  
)

# Execute the search with EarlyStopping
tuner.search(
    X_train, y_train,  
    epochs=100,
    batch_size=32,
    validation_data=(X_val, y_val),  
    verbose=2,
    callbacks=[early_stopping]
)

# Get the best model
best_hps = tuner.get_best_hyperparameters()[0]
best_model = tuner.hypermodel.build(best_hps)

print(best_model.summary())


Trial 30 Complete [00h 00m 14s]
val_accuracy: 0.520900309085846

Best val_accuracy So Far: 0.5659164190292358
Total elapsed time: 00h 07m 10s


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 3, 23, 95)      │           950 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 1, 8, 95)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 1, 8, 95)       │        81,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 1, 3, 95)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1, 3, 115)      │        11,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape_1 (Reshape)             │ (None, 3, 115)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 30)             │        17,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 30)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 110,861 (433.05 KB)

 Trainable params: 110,861 (433.05 KB)

 Non-trainable params: 0 (0.00 B)

None


In [2]:
# Fit the model n times and averages the metrics
n = 30
accuracy_list = []
f1_list = []
precision_list = []
recall_list = []

for _ in range(n):
    best_hps = tuner.get_best_hyperparameters()[0]
    best_model = tuner.hypermodel.build(best_hps)
    print("iteration: ", _)
    history = best_model.fit(
        X_train_full, y_train_full,
        epochs=250,
        batch_size=32,
        verbose=0
    )
    predictions = best_model.predict(X_test)
    test_pred = (predictions > 0.5).astype(int)

    accuracy = accuracy_score(y_test, test_pred)
    f1 = f1_score(y_test, test_pred)
    precision = precision_score(y_test, test_pred)
    recall = recall_score(y_test, test_pred)

    accuracy_list.append(accuracy)
    f1_list.append(f1)
    precision_list.append(precision)
    recall_list.append(recall)

# Calculate the average metrics
average_accuracy = np.mean(accuracy_list)
average_f1 = np.mean(f1_list)
average_precision = np.mean(precision_list)
average_recall = np.mean(recall_list)
    
# Print the average metrics
print('Accuracy:', average_accuracy)
print('F1:', average_f1)
print('Precision:', average_precision)
print('Recall:', average_recall)

iteration:  0
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
iteration:  1


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
iteration:  2


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
iteration:  3


...

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
iteration:  28


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
iteration:  29


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Accuracy: 0.5072463768115943
F1: 0.5256963190118714
Precision: 0.5125235847836358
Recall: 0.6371428571428572


In [9]:
# Seed 
seed_value = 1234
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Train the final model 
final_model = tuner.hypermodel.build(best_hps)
history = final_model.fit(
    X_train_full, y_train_full,
    epochs=250,
    batch_size=32,
    verbose=2
    )

# Evaluate the final model on the test data
predictions = final_model.predict(X_test)
test_pred = (predictions > 0.5).astype(int)

final_accuracy = accuracy_score(y_test, test_pred)
final_f1 = f1_score(y_test, test_pred)
final_precision = precision_score(y_test, test_pred)
final_recall = recall_score(y_test, test_pred)

# Print the final metrics
print('Final Model Metrics:')
print('Accuracy:', final_accuracy)
print('F1:', final_f1)
print('Precision:', final_precision)
print('Recall:', final_recall)

# Save the model
final_model.save('best_CNNLSTM3DCLASS.keras')

/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/250
98/98 - 2s - 17ms/step - accuracy: 0.5278 - loss: 0.6927
Epoch 2/250
98/98 - 0s - 3ms/step - accuracy: 0.5278 - loss: 0.6921
Epoch 3/250
98/98 - 0s - 4ms/step - accuracy: 0.5278 - loss: 0.6916
Epoch 4/250
98/98 - 0s - 4ms/step - accuracy: 0.5278 - loss: 0.6916
Epoch 5/250
98/98 - 0s - 3ms/step - accuracy: 0.5278 - loss: 0.6910
Epoch 6/250
98/98 - 0s - 4ms/step - accuracy: 0.5278 - loss: 0.6912
Epoch 7/250
98/98 - 0s - 4ms/step - accuracy: 0.5304 - loss: 0.6905
Epoch 8/250
98/98 - 0s - 3ms/step - accuracy: 0.5329 - loss: 0.6901
Epoch 9/250
98/98 - 0s - 4ms/step - accuracy: 0.5371 - loss: 0.6895
Epoch 10/250
98/98 - 0s - 3ms/step - accuracy: 0.5390 - loss: 0.6896
Epoch 11/250
98/98 - 0s - 3ms/step - accuracy: 0.5332 - loss: 0.6885
Epoch 12/250
98/98 - 0s - 3ms/step - accuracy: 0.5419 - loss: 0.6895
Epoch 13/250
98/98 - 0s - 3ms/step - accuracy: 0.5451 - loss: 0.6878
Epoch 14/250
98/98 - 0s - 3ms/step - accuracy: 0.5438 - loss: 0.6873
Epoch 15/250
98/98 - 0s - 4ms/step - accur